In [1]:
import numpy as np
import pandas as pd
import csv
from collections import namedtuple
from collections import defaultdict
import math
import time

In [2]:
df = pd.read_csv('nyc_taxi_data.csv')

df['pickup_longitude'] = df['pickup_longitude'].round(4)

df['pickup_latitude'] = df['pickup_latitude'].round(4)

df['dropoff_longitude'] = df['dropoff_longitude'].round(4)

df['dropoff_latitude'] = df['dropoff_latitude'].round(4)

In [3]:
df = df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','trip_distance']]

In [4]:
df['from_route'] = df['pickup_longitude'].astype(str) +"-"+ df["pickup_latitude"].astype(str)
df['to_route'] = df['dropoff_longitude'].astype(str) +"-"+ df["dropoff_latitude"].astype(str)

node_dict = {}

for i in range (len(df)):
    if df['from_route'][i] in node_dict:
        pass
    else:
        node_dict[df['from_route'][i]] = i 
        
    if df['to_route'][i] in node_dict:
        pass
    else:
        node_dict[df['to_route'][i]] = i+ 168152

In [5]:
new_df = pd.DataFrame()
new_df['node'] = node_dict.values()

ll = pd.Series(node_dict.keys())
new_df['lat_long'] = node_dict.keys()

In [6]:
df['node_pickup'] = df['from_route'].map(new_df.set_index('lat_long')['node'])
df['node_dropoff'] = df['to_route'].map(new_df.set_index('lat_long')['node'])

In [7]:
df[['node_pickup', 'node_dropoff' ]].to_csv('edges.csv', index = False)

In [8]:
df1 = pd.DataFrame()

df2 = pd.DataFrame()

df1 [['node_id', 'lat', 'long']] = df[['pickup_longitude', 'pickup_latitude', 'node_pickup']]

df2 [['node_id', 'lat', 'long']] = df[['dropoff_longitude', 'dropoff_latitude', 'node_dropoff']]


frames = [df1, df2]

nodes = pd.concat(frames)

nodes.to_csv('nodes.csv', index = False)

In [9]:
a  = list(df[['node_pickup', 'node_dropoff', 'trip_distance']].values)

from collections import defaultdict

def generateDirectedGraph(edges):
    graph = defaultdict(dict)
    for u, v, dist in edges:
        graph[u][v] = dist
        graph[v][u] = dist
    return graph
  
weighted_edges = a
d_wt_grph = generateDirectedGraph(weighted_edges)

In [10]:
d_wt_grph

defaultdict(dict,
            {0.0: {168152.0: 2.4,
              184801.0: 1.7,
              187068.0: 0.82,
              188049.0: 3.1,
              265554.0: 1.0},
             168152.0: {0.0: 2.4,
              34019.0: 1.28,
              225644.0: 2.08,
              174395.0: 1.63},
             1.0: {168153.0: 5.0,
              170466.0: 6.4,
              1675.0: 1.95,
              12077.0: 0.38,
              175760.0: 2.6,
              172533.0: 4.8,
              16599.0: 0.9,
              213769.0: 3.04,
              8649.0: 3.74,
              182317.0: 3.6,
              304137.0: 0.8,
              93450.0: 1.66},
             168153.0: {1.0: 5.0},
             2.0: {168154.0: 10.25, 182548.0: 20.24, 29323.0: 3.7},
             168154.0: {2.0: 10.25,
              15480.0: 9.8,
              25627.0: 9.74,
              3166.0: 11.95,
              268525.0: 11.2,
              1609.0: 7.6,
              295914.0: 2.3,
              211883.0: 9.7,
              

In [14]:
start = time.time()
def djkss(graph, pickup):
    q = [pickup]
    distsmin = {v: 10000000 for v in graph}
    distsmin[pickup] = 0
    prevs = {}

    while q:
        nod_pointer = q.pop(0)
        for nbr in graph[nod_pointer]:
            
            newDist = distsmin[nod_pointer] + graph[nod_pointer][nbr]
            
            if newDist < distsmin[nbr]:
                distsmin[nbr] = min(newDist, distsmin[nbr])
                q.append(nbr)
                prevs[nbr] = nod_pointer

    return distsmin, prevs

shortest_path_cost, prevs = djkss(d_wt_grph, 3)

In [15]:
def UniformCostSearch(graph, pickup, dropoff):
    distsmin, prevs = djkss(graph, pickup)
   
    
    path = []
    nod_pointer = dropoff
    while nod_pointer != pickup:
        if nod_pointer not in prevs:
            print("Path not reachable")
            break
        else:
            path.insert(0, nod_pointer)
            nod_pointer = prevs[nod_pointer]
    path.insert(0, pickup)
    
    if dropoff in distsmin and distsmin[dropoff] != float("inf"):
        print('cost bwtwn source to dropoff ' + str(distsmin[dropoff]))
        print('And the path between is ' + str(path))
        
UniformCostSearch(d_wt_grph, 2, 50)
end = time.time()
print(end - start)

cost is 13.130000000000003
And the path is [2, 29323.0, 207112.0, 198742.0, 171011.0, 2859.0, 453.0, 25494.0, 56100.0, 199132.0, 193120.0, 172307.0, 50]
10.508140087127686
